# Урок 1. Алгоритм линейной регрессии. Градиентный спуск

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14,
                     'xtick.labelsize': 14})

In [ ]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2]])
X

In [ ]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2) # <=> 1/n * np.sum((y_pred - y)**2)
    return err

In [ ]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

## Домашнее задание:

### Задание 1<br>
Подберите скорость обучения (alpha) и количество итераций для градиентного спуска (до совпадения ответов с результатами МНК). Как влияют друг на друга эти два параметра?

In [ ]:
n = X.shape[0]
alpha = 1e-3
w = np.array([1, 0.5])

for i in range(1000+1):
    y_pred = np.dot(w, X)
    err = calc_mse(y, y_pred)
    for j in range(w.shape[0]):
        w[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))
    if i % 100 == 0:
        print(i, w, err) # МНК: array([47.23214286,  3.91071429]) 45.937499999999986

__Решение:__<br>

In [ ]:
n = X.shape[1]
alpha = 1e-3
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')

for i in range(10000+1):
    y_pred = np.dot(W, X)
    err = calc_mse(y, y_pred)
    for j in range(W.shape[0]):
        W[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))
    if i % 1000 == 0:
        print(f'Weights = {W}, количество итераций = {i}. MSE: {err}')

Для решения данной задачи можно реализовать механизм перебора различных коэффициентов и сравнение получемой ошибки. Для этого код градиентного спуска нужно обернуть в функцию:

In [ ]:
def gradient_descent(alpha, iterations, W, X=X, y=y):
    n = X.shape[1]
    err = np.inf
    for i in range(iterations):
        y_pred = np.dot(W, X)
        err_new = calc_mse(y, y_pred)
        if err_new < err:
            err = err_new
            for j in range(W.shape[0]):
                W[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y))) 
        else:
            return i-1, W, err
    return i, W, err

Далее перебираем параметры с помощью двух циклов ```for ... in```:

In [ ]:
%%time
min_err = np.inf
min_alpha = ()
min_iter = ()

for alpha in [1e-1, 1e-2, 1e-3, 1e-4]:
    for iters in [1e3, 1e4, 1e5, 1e6]:
        W = np.array([1, 0.5])
        num, wi, err = gradient_descent(alpha, int(iters), W)
        print(f'Скорость обучения (альфа) = {alpha} | количество итераций = {iters}\n'
        f'MSE: {err} | Подобранные веса = {W}\n')
        if err < min_err:
            min_err = err
            min_alpha = alpha
            min_iters = iters

print(f'\n\nМинимальная ошибка {min_err} достигается при скорости обучения (альфа) = {min_alpha} и количестве итераций = {min_iters}')

__Ответ:__<br>
Минимальная ошибка $MSE = 43.968750000001044$ достигается при скорости обучения (альфа) $\alpha = 0.01$ и количестве итераций $ Iterations = 10 000$. Скорость обучения значительно увеличивает количество итераций необходимое для вычисления минимальной ошибки. При $\alpha = 0.0001$ необходимое количество итераций более $10^7$, что ведет к повышению стоимости вычислений.

### $^*$Задание 2<br>
В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее:

In [ ]:
w = np.array([1, 0.5])

for i in range(1000):
    y_pred = np.dot(w, X)
    err = calc_mse(y, y_pred)
    '''for j in range(w.shape[0]):
        w[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))'''
    w -= alpha * (1/n * 2 * np.sum(X * (y_pred - y)))
    if i % 100 == 0:
        print(i, w, err) # [47.23214286  3.91071429] 45.937499999999986

__Решение:__<br>
Ошибка заключается в суммировании всех элементов врезультате матричного умножения и выдаче скаляра, который затем расширяется до вектора с одинаковыми элементами. Исправляется путем указания столбца по которому вести расчет:

In [ ]:
w = np.array([1, 0.5])

for i in range(1000):
    y_pred = np.dot(w, X)
    err = calc_mse(y, y_pred)
    '''for j in range(w.shape[0]):
        w[j] -= alpha * (1/n * 2 * np.sum(X[j] * (y_pred - y)))'''
    w -= alpha * (1/n * 2 * np.sum(X * (y_pred - y),  axis=1))
    if i % 100 == 0:
        print(i, w, err) # [47.23214286  3.91071429] 45.937499999999986

### $^*$Задание 3<br>
Вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию начинает изменяться ниже определенного порога. 

Сколько нужно сделать итераций, если установить допустимое отклонение mse в размере $\text{diff}=10^{-6}$, а значение $\alpha=10^{-2}$?

__Решение:__<br>
Упрощенный аналог параметра ```tol``` в линейной регрессии в ```sklearn```:

In [116]:
w = np.array([1, 0.5])
diff = 1e-6
alpha = 1e-2

err_min = np.inf
err = 1e6
i = 0
while err_min - err > diff:
    y_pred = np.dot(w, X)
    err, err_min = calc_mse(y, y_pred), err
    w -= (alpha * (1/n * 2 * np.sum(X * (y_pred - y), axis=1)))
    i += 1
print(f'Итераций = {i}, Скорость обучения 𝛼 = {alpha}, Подобранные веса = {w}, MSE = {err}')

Итераций = 905, Скорость обучения 𝛼 = 0.01, Подобранные веса = [45.05195252  3.81441262], MSE = 43.96880336630431


__Ответ:__<br>
При $\text{diff}=10^{-6}$ и $\alpha=10^{-2}$ нужно $905$ итераций.

Заметки по матричным вычислениям http://www.machinelearning.ru/wiki/images/2/2a/Matrix-Gauss.pdf